<a href="https://colab.research.google.com/github/ETGrif/ecolabel-translator/blob/main/LLM_Front_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Structure

In [ ]:
/eco_label_translator
    ├── app.py               # Main Flask application
    ├── templates/
    │   ├── home.html        # Homepage template
    │   ├── results.html     # Results page template
    │   └── chat.html        # Chat interface template
    ├── static/
    │   └── style.css        # CSS for styling



app.py

This is the main Flask application that will handle the routes for the front end

* init_chat: Initializes a chat session with ChatManager and redirects to the chat interface.
* chat: Displays the chat interface and handles messages. It uses ChatManager to get responses.
* terminate_chat: Ends the chat session and redirects to the homepage.





In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import requests  # For making HTTP requests to backend API
from some_module import ChatManager, DBManager, GPTManager  # Assuming these are implemented in some_module

app = Flask(__name__)

# Initializing managers
db_manager = DBManager()
chat_manager = ChatManager()
gpt_manager = GPTManager()

@app.route('/')
def home():
    """Renders the homepage with a search form and a button to initiate chat."""
    return render_template('home.html')

@app.route('/results', methods=['GET'])
def results():
    """Handles the results page that shows the eco-label information."""
    label = request.args.get('label')  # Gets the 'label' query parameter from the URL
    if not label:
        return redirect(url_for('home'))  # Redirects to home if no label is provided

    # Send a GET request to the backend API to get eco-label information
    try:
        response = requests.get(f'http://localhost:5000/search?label={label}')
        response.raise_for_status()  # Raises an error for unsuccessful status codes
        data = response.json()  # Parses JSON response
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        data = None  # If an error occurs, set data to None

    return render_template('results.html', label=label, data=data)

@app.route('/init_chat', methods=['POST'])
def init_chat():
    """Initializes a new chat session and redirects to the chat page."""
    chat_id = chat_manager.create_chat()  # Initializes a chat session in ChatManager
    return redirect(url_for('chat', chat_id=chat_id))

@app.route('/chat/<chat_id>', methods=['GET', 'POST'])
def chat(chat_id):
    """Handles chat interface for ongoing chat sessions."""
    if request.method == 'POST':
        message = request.form.get('message')  # User's message from the form
        response = chat_manager.get_resp(chat_id, message)  # Get response from ChatManager
        return render_template('chat.html', chat_id=chat_id, messages=response['messages'])
    return render_template('chat.html', chat_id=chat_id)

@app.route('/terminate', methods=['POST'])
def terminate_chat():
    """Terminates an ongoing chat session and redirects to the homepage."""
    chat_id = request.form.get('chat_id')
    chat_manager.terminate_chat(chat_id)  # Terminates chat in ChatManager
    return redirect(url_for('home'))

if __name__ == '__main__':
    app.run(debug=True)


home.html

The HTML template for the homepage where users enter the eco-label name

* A form is added to initiate a chat session, which posts to /init_chat and redirects to the chat page


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Eco-Label Translator</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Eco-Label Translator</h1>
        <form action="{{ url_for('results') }}" method="get">
            <input type="text" name="label" placeholder="Enter eco-label name" required>
            <button type="submit">Search</button>
        </form>

        <!-- Form to start a new chat session -->
        <form action="{{ url_for('init_chat') }}" method="post">
            <button type="submit">Start a Chat</button>
        </form>
    </div>
</body>
</html>

results.html

The HTML template for the results page, which displays the fetched eco-label data

* A form for fetching a detailed explanation from GPTManager is included, submitting the label as a hidden field

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Results for {{ label }}</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Results for "{{ label }}"</h1>

        {% if data %}
            <div class="results">
                <p><strong>Description:</strong> {{ data['description'] }}</p>
                <p><strong>EPA Recommendation:</strong> {{ data['epaRecommendation'] }}</p>

                <!-- Form to get a detailed explanation from GPTManager -->
                <form action="{{ url_for('get_resp') }}" method="post">
                    <input type="hidden" name="label" value="{{ label }}">
                    <button type="submit">Get Detailed Explanation</button>
                </form>
            </div>
        {% else %}
            <p>No data found for this label.</p>
        {% endif %}

        <a href="{{ url_for('home') }}" class="back-link">Go back</a>
    </div>
</body>
</html>

chat.html

The chat interface, allowing users to engage in a conversation with the backend

* Displays chat messages, including user and assistant responses
* The form allows users to submit messages to continue the conversation
* The "End Chat" button posts to /terminate, ending the chat session



In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chat Assistant</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h2>Eco-Label Chat Assistant</h2>
        <div id="messages">
            {% for message in messages %}
                <p><strong>{{ message['sender'] }}:</strong> {{ message['text'] }}</p>
            {% endfor %}
        </div>
        <form action="{{ url_for('chat', chat_id=chat_id) }}" method="post">
            <input type="text" name="message" placeholder="Type your message..." required>
            <button type="submit">Send</button>
        </form>
        <form action="{{ url_for('terminate') }}" method="post">
            <input type="hidden" name="chat_id" value="{{ chat_id }}">
            <button type="submit">End Chat</button>
        </form>
    </div>
</body>
</html>


style.css

The CSS file to style the front end

In [ ]:
/* style.css */

body {
    font-family: Arial, sans-serif;
    text-align: center;
    margin: 0;
    padding: 0;
    background-color: #f9f9f9;
}

.container {
    max-width: 600px;
    margin: 50px auto;
    padding: 20px;
    border-radius: 8px;
    background-color: #ffffff;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
}

h1, h2 {
    color: #2c3e50;
}

input[type="text"], button {
    padding: 10px;
    margin-top: 10px;
    width: 80%;
    border: 1px solid #ddd;
    border-radius: 4px;
}

button {
    width: auto;
    background-color: #2980b9;
    color: white;
    cursor: pointer;
    border: none;
    margin-top: 10px;
}

button:hover {
    background-color: #3498db;
}

.results, #messages {
    text-align: left;
    margin-top: 20px;
}

.back-link, .end-chat {
    color: #2980b9;
    text-decoration: none;
    margin-top: 20px;
    display: inline-block;
}

.back-link:hover, .end-chat:hover {
    color: #3498db;
}
